# Prvi susret sa modelima mašinskog učenja u praksi

Osnovni zadatak mašinskog učenja je da u velikom broju tabelarno struktuiranih podataka pronađe određene zakonitosti - trendove rasta, opadanja, grupisanja, uzajamne korelacije podataka u različitim kolonama i vrstama. Algoritmi mašinskog učenja su sposobni da uoče ove zakonitosti, a da ih potom iskoriste da za nove, nepotpune podatke predvide nedefinisane vrednosti ili te nove podatke dodele nekoj grupi ili kategoriji.

Uočavanje zakonitosti algoritmi vrše na osnovu grupa potpunih podataka. Ove podatke nazivamo potpunim jer sadrže:

- Opažaje ili obzervacije. Ove podatke nazivamo tzv. ulaznim podacima.
- Ishode analize ovih opažaja. Ove podatke nazivamo tzv. izlaznim podacima

Proces u okviru kojeg algoritmi uočavaju zakonitosti nazivamo učenjem ovih algoritama, odnosno mašinskim učenjem. Aktivnost u okviru koje naučeni algoritam predviđa ishod analize, odnosno izlazni podatak za nepotpune (ulazne) podatke (nepotpuni su jer im nedostaje izlazni podatak), nazivamo predviđanjem. Naučeni algoritam nazivamo modelom mašinskog učenja u užem smislu.

Svi ovi podaci su smešteni u tabelarnoj strukturi. Jedna vrsta ove strukture je skup podataka o raznim opažajima (ulazni podatak) i jedan ili više podataka koji se smatraju analizom podataka o opažajima (izlazni podatak). Jedna kolona ove strukture skladišti sve podatke jedne veličine - svi ovi podaci su iste prirode, pa i istog tipa.

Cilj ove vežbe je da sa minimalnim korišćenjem Python instrukcija demonstrira osnovne faze procesa izrade modela mašinskog učenja i predviđanja na osnovu nepotpunih podataka.

## Vrste problema koji se rešavaju mašinskim učenjem

Problem predviđanja podataka se rešava tzv. nadgledanim mašinskim učenjem. Nadgledano mašinsko učenje podrazumeva da postoji potpuni skup podataka (odnosno, skup koji sadrži poznate ulazne i izlazne podatke) na osnovu kojeg se vrši učenje. Ipak, do ovakvih podataka je često veoma teško doći. U situaciji kada su skupovi nepotpuni, odnosno, kada nam izlazni podaci nisu poznati, predviđanje nije moguće. Ipak, moguće je uočavanje trendova i grupa srodnih podataka. Problem grupisanja podataka se rešava primenom tzv. nenadgledanog mašinskog učenja.

Najčešće grupe problema koji se rešavaju nadgledanim mašinskim učenjem (uobičajeno, problemi se svode na ove grupe problema) su klasifikacija i regresija. Izbor grupe zavisi od karaktera veličine koja se predviđa. Ukoliko je ona nominalna (ili ordinalna), odnosno postoji konačan (tipično mali) broj vrednosti koje ta veličina može da ima, problem predviđanja je problem klasifikacije. Ukoliko postoje samo dve vrednosti nominalne veličine, radi se o problemu binarne klasifikacije. Ukoliko ih ima više, nazivamo ga problemom višestruke klasifikacije. Ukoliko je veličina koja se predviđa kontinualna (realan broj), problem predviđanja je problem regresije.

## Predviđanje verovatnoće dijagnoze dijabetesa, na osnovu opažanja prikupljenih tokom dijagnostičkog procesa

Modeli mašinskog učenja imaju veliku primenu u medicini, naročito u procesu dijagnoze bolesti na osnovu prikupljenih ili izmerenih fizioloških parametara. U okviru ove vežbe, demonstriraju se osnovne faze izrade modela mašinskog učenja za dijagnozu dijabetesa, na osnovu sledećih poznatih podataka o pacijentu: broj trudnoća, koncentracija šećera u krvi, izmereni krvni pritisak, debljina kože, insulin, indeks telesne mase i starost. 

Za izradu modela mašinskog učenja se koriste poznati podaci o 767 pacijenata ženskog pola.

### Uvoz postojećih poznatih podataka za učenje

Poznati podaci se nalaze u csv (Comma Separated Values) datoteci. Csv format omogućava skladištenje tabelarnih struktura u običnim tekstualnim datotekama. Ovakav format podrazumeva da se podaci iz jedne vrste nalaze u jednom redu u datoteci, a da su podaci iz pojedinačnih kolona jedne vrste odvojeni tzv. separatorom (uobičajeni karakter koji se koristi kao separator je zarez).
Otvaranje skupa podataka koji se nalazi u csv datoteci se vrši pozivanjem komande read_csv pandas paketa. Rezultat izvršenja ove komande je DataFrame struktura sa podacima učitanim iz csv datoteke. Nakon kreiranja strukture, njoj se definišu i nazivi kolona.

In [1]:
import pandas as pd
df=pd.read_csv('kurs_diabetes.csv')
df.columns=['preg','plas','pres','skin','insu','mass','pedi','age','class']

Delimični prikaz podataka učitanih u DataFrame strukturu se vrši pozivanjem funkcije head ove strukture. Ukoliko se ne navede broj redova koji se prikazuje, podrazumevani broj je 5.

In [2]:
df.head()

,preg,plas,pres,skin,insu,mass,pedi,age,class
0,1,85,66,29,0,26.6,0.351,31,tested_negative
1,8,183,64,0,0,23.3,0.672,32,tested_positive
2,1,89,66,23,94,28.1,0.167,21,tested_negative
3,0,137,40,35,168,43.1,2.288,33,tested_positive
4,5,116,74,0,0,25.6,0.201,30,tested_negative


Kao što se vidi, strukturu čine 8 tzv. ulaznih podataka, odnosno opažanja (preg, plas, pres, skin, insu, mass, pedi, age) i jedan izlazni podatak, odnosno rezultat analize ovih opažanja (class).

Sada je očigledno da se problem koji rešavamo svodi na to da se na osnovu 8 ulaznih podataka koje je relativno lako uočiti za nepoznatog pacijenta kroz proces laboratorijske analize, može predvideti da li pacijent ima dijabetes ili ne.

### Kreiranje skupova ulaznih i izlaznih podataka

Prvi korak u kreiranju modela mašinskog učenja je kreiranje skupova ulaznih i izlaznih podataka na osnovu strukture koju čine učitani podaci. Kreiraju se dva niza i to isecanjem iz niza u koji je funkcijom 'values' transformisan DataFrame df.

In [3]:
arr=df.values
X=arr[:,0:8]
y=arr[:,8]

### Kreiranje posebnih skupova za učenje i testiranje

Kao što je već pomenuto, ovaj skup podataka sadrži 767 vrsti. Da bi mogli da izmerimo tačnost predviđanja modela, potrebno je da jedan broj tih vrsti "proglasimo" nepotpunim podacima. Te "nepotpune" podatke možemo da koristimo za testiranje tačnosti modela. Naime, za svaki od ulaznih podataka ovog podskupa, kreirani model će izvršiti predviđanje i to predviđanje uporediti sa tačnim izlaznim podatkom. Procenat tačnih predviđanja ćemo nazvati tačnošću modela.

Podelu celog skupa na podskup za učenje (70% svih podataka) i podskup za testiranje (30%) vrši funkcija train_test_split paketa model_selection koji je deo paketa sklearn.

In [4]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test=train_test_split(X,y,test_size=0.3, random_state=7)

### Učenje izabranog algoritma i procena tačnosti

Sada, možemo da izvršimo učenje algoritma. Prethodno, potrebno je izabrati algoritam mašinskog učenja. Postoji veliki broj algoritama koji se može naučiti. Svaki od njih karakteriše metoda predviđanja, određeni broj pretpostavki (aproksimacija) o podacima, itd. Za potrebe ove demonstracije, izabrani algoritam je logistička regresija. Model logističke regresije se učitava i uči podacima za učenje.

In [5]:
from sklearn.linear_model import LogisticRegression
model=LogisticRegression()
model.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

Kao što je već pomenuto, testiranje modela se vrši korišćenjem podskupa za testiranje.

In [6]:
accuracy=model.score(X_test, y_test)
print('Tačnost modela je:'+str(accuracy))

Tačnost modela je:0.7489177489177489


Parametar tačnosti se interpretira na sledeći način: Ukoliko na osnovu poznatih ulaznih podataka za novog pacijenta algoritam utvrdi da on ima ili nema dijabetes, verovatnoća da je njegovo predviđanje tačno je 74.89%.

### Kako se kreirani model mašinskog učenja koristi u praksi?

Iako je tačnost predviđanja solidna, napominje se da je ovaj primer pojednostavljen sa ciljem konkretne i kratke demonstracije procesa mašinskog učenja. Modeli mašinskog učenja se u praksi kreiraju uzimajući u obzir i analizu podataka, kreiranje novih veličina od postojećih, izbor i podešavanje algoritma, transformaciju podataka u oblik pogodan za tumačenje od strane izabranog algoritma itd. Na ovaj način, tačnost modela se uvećava.

Model koji prilikom testiranja pokazuje najveću tačnost se bira kao onaj koji će se koristiti u budućnosti za predviđanje dijagnoze za novopristiglog pacijenta. Jednom naučeni model se može snimiti u posebnoj datoteci i koristiti kasnije, za predviđanje.

In [7]:
import pickle
pickle.dump(model, open('model_diabetes.sav','wb'))

Prilikom pristizanja novog pacijenta, on će biti upućen na laboratorijske analize. One će rezultovati sa novim ulaznim podacima za tog pacijenta. Na primer,

In [8]:
xnovi=[[2,197,70,45,543,30.5,0.158,53]]

Napomena: dvostruke zagrade u deklaraciji promenljive xnovi se koriste jer funkcija za predviđanje očekuje ne jedan već više podataka. Tako, formira se dvodimenzionalni niz, ali samo sa jednom vrstom.

Za ovaj, novi skup ulaznih podataka, program za predviđanje će učitati datoteku sa modelom, predvideti i prikazati dijagnozu.

In [9]:
mod=pickle.load(open('model_diabetes.sav','rb'))
ynovi=mod.predict(xnovi)
print(ynovi)

['tested_positive']
